<a href="https://colab.research.google.com/github/December04/MachineLearningProgramming/blob/main/Week10/Assignment04.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

CNN LSTM

In [17]:
import numpy as np
import pandas as pd
import os
import tensorflow as tf
from tensorflow.keras import layers, models

In [19]:
HAR_PATH = "/content/drive/MyDrive/Colab Notebooks/UCI HAR Dataset/UCI HAR Dataset"

SIGNALS = [
    "total_acc_x","total_acc_y","total_acc_z",
    "body_acc_x","body_acc_y","body_acc_z",
    "body_gyro_x","body_gyro_y","body_gyro_z"
]

In [20]:
def load_signals(group):
    loaded = []

    for name in SIGNALS:
        filepath = f"{HAR_PATH}/{group}/Inertial Signals/{name}_{group}.txt"

        if not os.path.exists(filepath):
            raise FileNotFoundError(f"❌ 파일 없음: {filepath}")

        data = pd.read_csv(filepath, sep=r"\s+", header=None).values
        loaded.append(data)

    # (9, n_samples, 128) → (n_samples, 128, 9)
    return np.transpose(np.asarray(loaded), (1, 2, 0))

In [21]:
def load_labels(group):
    filepath = f"{HAR_PATH}/{group}/y_{group}.txt"

    if not os.path.exists(filepath):
        raise FileNotFoundError(f"❌ 레이블 파일 없음: {filepath}")

    labels = pd.read_csv(filepath, header=None)[0].values - 1   # 1→0, 2→1 ...
    return labels


In [22]:
X_train = load_signals("train")
X_test = load_signals("test")
y_train = load_labels("train")
y_test = load_labels("test")

print("데이터 로드 완료")
print("X_train:", X_train.shape)
print("X_test :", X_test.shape)
print("y_train:", y_train.shape)
print("y_test :", y_test.shape)

데이터 로드 완료
X_train: (7352, 128, 9)
X_test : (2947, 128, 9)
y_train: (7352,)
y_test : (2947,)


In [23]:
assert X_train.shape[1:] == (128, 9), "❌ 입력 shape 오류!"
assert X_test.shape[1:] == (128, 9), "❌ 입력 shape 오류!"

In [24]:
def create_cnn_lstm_best(input_shape, num_classes):
    model = models.Sequential([
        # CNN Feature Extractors
        layers.Conv1D(64, 3, activation='relu', padding='same', input_shape=input_shape),
        layers.BatchNormalization(),
        layers.MaxPooling1D(2),

        layers.Conv1D(128, 3, activation='relu', padding='same'),
        layers.BatchNormalization(),
        layers.MaxPooling1D(2),   # 128→64→32 timesteps

        # LSTM Sequence Learner
        layers.LSTM(128, return_sequences=True),
        layers.LSTM(64, return_sequences=False),

        # Dense Classifier
        layers.Dense(64, activation='relu'),
        layers.Dropout(0.4),
        layers.Dense(num_classes, activation='softmax')
    ])
    return model

In [25]:
model = create_cnn_lstm_best((128, 9), 6)
model.compile(
    optimizer=tf.keras.optimizers.Adam(0.0005),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

model.summary()

/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d_4 (Conv1D)               │ (None, 128, 64)        │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_4           │ (None, 128, 64)        │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_4 (MaxPooling1D)  │ (None, 64, 64)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_5 (Conv1D)               │ (None, 64, 128)        │        24,704 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_5           │ (None, 64, 128)        │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_5 (MaxPooling1D)  │ (None, 32, 128)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_4 (LSTM)                   │ (None, 32, 128)        │       131,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_5 (LSTM)                   │ (None, 64)             │        49,408 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 64)             │         4,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 6)              │           390 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 212,806 (831.27 KB)

 Trainable params: 212,422 (829.77 KB)

 Non-trainable params: 384 (1.50 KB)

In [26]:
history = model.fit(
    X_train, y_train,
    validation_split=0.2,
    epochs=30,
    batch_size=64,
    verbose=1
)

Epoch 1/30
92/92 ━━━━━━━━━━━━━━━━━━━━ 25s 196ms/step - accuracy: 0.6508 - loss: 0.9564 - val_accuracy: 0.1883 - val_loss: 2.2383
Epoch 2/30
92/92 ━━━━━━━━━━━━━━━━━━━━ 18s 192ms/step - accuracy: 0.9454 - loss: 0.1811 - val_accuracy: 0.2549 - val_loss: 2.1949
Epoch 3/30
92/92 ━━━━━━━━━━━━━━━━━━━━ 21s 193ms/step - accuracy: 0.9551 - loss: 0.1273 - val_accuracy: 0.6329 - val_loss: 1.2485
Epoch 4/30
92/92 ━━━━━━━━━━━━━━━━━━━━ 17s 183ms/step - accuracy: 0.9531 - loss: 0.1122 - val_accuracy: 0.8810 - val_loss: 0.3577
Epoch 5/30
92/92 ━━━━━━━━━━━━━━━━━━━━ 18s 198ms/step - accuracy: 0.9554 - loss: 0.1047 - val_accuracy: 0.8987 - val_loss: 0.2821
Epoch 6/30
92/92 ━━━━━━━━━━━━━━━━━━━━ 19s 181ms/step - accuracy: 0.9527 - loss: 0.1009 - val_accuracy: 0.9157 - val_loss: 0.3116
Epoch 7/30
92/92 ━━━━━━━━━━━━━━━━━━━━ 17s 182ms/step - accuracy: 0.9542 - loss: 0.1023 - val_accuracy: 0.9381 - val_loss: 0.2591
Epoch 8/30
92/92 ━━━━━━━━━━━━━━━━━━━━ 17s 185ms/step - accuracy: 0.9555 - loss: 0.0993 - val_accu

In [27]:
loss, acc = model.evaluate(X_test, y_test, verbose=0)
print(f"\nCNN LSTM Test Accuracy: {acc:.4f}")
print(f"Test Loss: {loss:.4f}")


CNN LSTM Test Accuracy: 0.9267
Test Loss: 0.3514
